In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
SELECT *
FROM CREDITGENIE.DDB_PROD.PLAIDTRANSACTION 
WHERE plaidaccountid IN (select plaidaccountid from creditgenie.ddb_prod.decision where cast(createdat as date) = '2025-04-01' and iseligible = True limit 100)
and TRANSACTIONDATE > '2025-01-01'
and is_pending = FALSE
order by transactiondate desc

In [ ]:
df = cell2.to_pandas()
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = -df[numeric_cols]

In [ ]:
import pandas as pd
import snowflake.snowpark.functions as F
from snowflake.snowpark import Session
result3 = df['PLAIDACCOUNTID']

session = get_active_session()

plaidaccountid_list = result3.tolist()
plaidaccountid_str = ', '.join(f"'{id}'" for id in plaidaccountid_list)

query = f'''
select plaidaccountid, AVAILABLE, "current", createdat from creditgenie.ddb_prod.plaidaccountbalance b
where b.plaidaccountid in ({plaidaccountid_str})
and b.createdat between '2025-01-01' and '2025-05-15'
'''

result_df3 = session.sql(query)
dft = result_df3.collect()
dft = pd.DataFrame(dft)
dft.columns = [column.lower() for column in dft.columns]

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import optuna
from sklearn.metrics import mean_squared_error

# --- Constants ---
PREDICTION_START_DATE = pd.to_datetime("2025-04-05")
MAX_PREDICTION_DATE = pd.to_datetime("2025-05-02")
your_threshold = -100

# --- Tune Prophet hyperparameters using Optuna on one sample ID ---
sample_id = df["PLAIDACCOUNTID"].unique()[0]
group = df[df["PLAIDACCOUNTID"] == sample_id].copy()

# Use transaction data for sample tuning
data = group.copy()
data["DATE"] = pd.to_datetime(data["TRANSACTIONDATE"])
data = data.groupby("DATE").sum(numeric_only=True).reset_index()
data.rename(columns={"DATE": "ds", "TRANSACTIONAMOUNT": "y"}, inplace=True)

# Restrict to training data only (before 06-05)
data = data.sort_values("ds")
train_df_sample = data[data["ds"] <= pd.to_datetime("2025-04-04")]

def objective(trial):
    changepoint_prior_scale = trial.suggest_float("changepoint_prior_scale", 0.001, 0.5, log=True)
    seasonality_prior_scale = trial.suggest_float("seasonality_prior_scale", 0.01, 10.0, log=True)
    seasonality_mode = trial.suggest_categorical("seasonality_mode", ["additive", "multiplicative"])

    try:
        model = Prophet(
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=False,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            seasonality_mode=seasonality_mode
        )
        model.fit(train_df_sample)

        future = model.make_future_dataframe(periods=30)
        forecast = model.predict(future)

        merged = train_df_sample.merge(forecast[["ds", "yhat"]], on="ds", how="left")

        holdout_size = int(len(merged) * 0.2)
        if holdout_size == 0:
            return float("inf")
        holdout = merged[-holdout_size:]

        rmse = np.sqrt(mean_squared_error(holdout["y"], holdout["yhat"]))
        return rmse

    except Exception:
        return float("inf")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
print("Best hyperparameters (from sample ID):", best_params)

# --- Final loop for all IDs ---
prophet_results = {}
forecast_dfs = {}

for plaid_id, group in df.groupby("PLAIDACCOUNTID"):
    # Get earliest balance from dft
    group_dft = dft[dft["plaidaccountid"] == plaid_id].copy()
    group_dft["createdat"] = pd.to_datetime(group_dft["createdat"]).dt.tz_localize(None)
    group_dft["balance_value"] = group_dft["current"].where(pd.notnull(group_dft["current"]), group_dft["available"])
    group_dft = group_dft.sort_values("createdat")

    if group_dft.empty or pd.isna(group_dft.iloc[0]["balance_value"]):
        continue

    earliest_balance_value = group_dft.iloc[0]["balance_value"]
    start_date = group_dft.iloc[0]["createdat"]

    # Filter transactions data from start_date
    data = group.copy()
    data["DATE"] = pd.to_datetime(data["TRANSACTIONDATE"])
    data = data[data["DATE"] >= start_date]

    if data.empty or len(data) < 20:
        continue

    # Aggregate daily, sort
    data = data.groupby("DATE").sum(numeric_only=True).reset_index()
    data.rename(columns={"DATE": "ds", "TRANSACTIONAMOUNT": "flow"}, inplace=True)
    data = data.sort_values("ds")

    # Create cumulative balance series starting from earliest_balance_value
    data["y"] = earliest_balance_value + data["flow"].cumsum()

    # Restrict to training data only for model fit
    train_df = data[data["ds"] <= pd.to_datetime("2025-04-04")]

    if train_df.empty or len(train_df) < 20:
        continue

    try:
        model = Prophet(
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=False,
            changepoint_prior_scale=best_params["changepoint_prior_scale"],
            seasonality_prior_scale=best_params["seasonality_prior_scale"],
            seasonality_mode=best_params["seasonality_mode"]
        )
        model.fit(train_df)

        num_test_days = (MAX_PREDICTION_DATE - PREDICTION_START_DATE).days + 1
        future = model.make_future_dataframe(periods=num_test_days)
        forecast = model.predict(future)

        # Filter only prediction window
        forecast_future = forecast[
            (forecast["ds"] >= PREDICTION_START_DATE) &
            (forecast["ds"] <= MAX_PREDICTION_DATE)
        ]

        # ✅ Save merged dataframe with ALL actual transactions
        actuals_df = data[["ds", "y"]]
        merged_df = forecast.merge(actuals_df, on="ds", how="left")
        forecast_dfs[plaid_id] = merged_df

        below_threshold = forecast_future["yhat"] < your_threshold

        if below_threshold.any():
            predicted_index = forecast_future[below_threshold]["ds"].iloc[0]

            if predicted_index < PREDICTION_START_DATE:
                predicted_index = PREDICTION_START_DATE
            if predicted_index > MAX_PREDICTION_DATE:
                predicted_index = MAX_PREDICTION_DATE
        else:
            predicted_index = pd.NaT

    except Exception:
        predicted_index = pd.NaT

    days_difference = pd.NA  # No actual_date logic now

    prophet_results[plaid_id] = {
        "predicted_date": predicted_index,
        "days_difference": days_difference,
        "source": "prophet"
    }

# --- Final results DataFrame ---
prophet_results_df = pd.DataFrame([
    {
        "plaidaccountid": pid,
        "predicted_date": res["predicted_date"],
        "days_difference": res["days_difference"],
        "source": res["source"]
    }
    for pid, res in prophet_results.items()
])

prophet_results_df


In [ ]:
import matplotlib.pyplot as plt

# --- Choose an ID from forecast_dfs to plot ---
chosen_id = list(forecast_dfs.keys())[31]  # Replace with index of ID

df_plot = forecast_dfs.get(chosen_id)

if df_plot is not None:
    # Filter to prediction window: start and end dates
    df_plot_filtered = df_plot[
        #(df_plot["ds"] >= PREDICTION_START_DATE) &
        (df_plot["ds"] <= MAX_PREDICTION_DATE)
    ]

    plt.figure(figsize=(12, 6))
    plt.plot(df_plot_filtered["ds"], df_plot_filtered["yhat"], label="Predicted balance", color="blue", linewidth=2)
    plt.plot(df_plot_filtered["ds"], df_plot_filtered["y"], label="Actual balance", color="black", linewidth=2, alpha=0.7)
    plt.axvline(PREDICTION_START_DATE, color="green", linestyle="--", label="Prediction Start")
    plt.axvline(MAX_PREDICTION_DATE, color="red", linestyle="--", label="Prediction End")
    plt.xlabel("Date")
    plt.ylabel("balance/Day")
    plt.title(f"Predicted vs Actual Balance for ID: {chosen_id}\n({PREDICTION_START_DATE.date()} to {MAX_PREDICTION_DATE.date()})")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print(f"No forecast data available for ID: {chosen_id}")


In [ ]:
# dft2 = dft[dft['plaidaccountid'] == 'ZVNQqV7boQHL8PE0DoZwf0AvojD3qpCxRqPBY']
# dft2

df2 = df[df['PLAIDACCOUNTID'] == 'ZVNQqV7boQHL8PE0DoZwf0AvojD3qpCxRqPBY']
df2
